In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import sys
import pickle
from itertools import product

import numpy as np
import pandas as pd
import scipy.stats as stats

from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns


sns.set_theme(style='whitegrid')
sns.set_theme(style='ticks')



In [ ]:
repo_dir = Path('../..')

In [ ]:
if str(repo_dir) not in sys.path:
    sys.path.append(str(repo_dir))
    
from analysis.curve_fitting.src.fitting_functions import LOSS_FUNCTIONS
from analysis.curve_fitting.src.utils import apply_filters, load_yaml, convert_loss_parameters, convert_loss_parameters_batch

from visualization.src.utils import COLOR_PALETTES, set_ticks, save_figs, COLORS
from visualization.src.visualize import plot_reg, plot_reg_bivariate, plot_confidence_intervals


In [ ]:
args = {
    'results_csv': repo_dir / 'results' / 'benchmark_scores.csv',
    'results_csv_ckpts': repo_dir / 'results' / 'benchmark_scores_ckpts.csv',
}

In [ ]:
results_csv = args['results_csv']
df_results = pd.read_csv(results_csv)

results_csv_ckpts = args['results_csv_ckpts']
df_results_ckpts = pd.read_csv(results_csv_ckpts)

## Experiment Configuration

In [ ]:
config = {
    'data_filters': {
        'set_filters': {
            'region': [
                'V1',
                'V2',
                'V4',
                'IT',
                'Behavioral'
                ],
            'dataset': [
                'imagenet',
                ],
            'arch': [
                'resnet18', 
                'resnet50', 
                'deit_small', 
                'deit_base', 
                'vit_small', 
                'vit_base',
                ],
            'samples_per_class': [0]
            },
            
    'boolean_filters': {
        'equals_false': [
            'is_pretrained',
            'is_random',
            # 'is_ssl',
            'is_ablation',
            'is_adv',
            ],
        },
    
    # 'group_by': {
    #     'avg_score': {
    #         'keys': [
    #             'model_id',
    #             'arch',
    #             'dataset',
    #             'flops',
    #             'n_params',
    #             'n_samples',
    #             'n_samples_seen',
    #             'total_flops',
    #             'arch_family',
    #             'samples_per_class',
    #             'adv_method',

    #         ],
    #         'reduce': {'score': 'mean'}}},

    'combine_arch_families': True,
    }
}

## Apply Data Filters

In [ ]:
df = apply_filters(df_results, config['data_filters'])

In [ ]:
model_sort = {
        'AlexNet': 0, 'CORnet-S':1, 'EfficientNet': 2, 'ConvNeXt': 3, 'ViT':4, 'ResNet':6, 'ResNetFlex':5
}
df['model_sort'] = df.arch_family.map(model_sort)
df.sort_values(['model_sort'], inplace=True, ascending=True)

In [ ]:
df.loc[df.arch=='deit_small', 'arch'] = 'ViT-S'
df.loc[df.arch=='vit_small', 'arch'] = 'ViT-S'
df.loc[df.arch=='deit_base', 'arch'] = 'ViT-B'
df.loc[df.arch=='vit_base', 'arch'] = 'ViT-B'
df.loc[df.arch=='resnet18', 'arch'] = 'ResNet-18'
df.loc[df.arch=='resnet50', 'arch'] = 'ResNet-50'

df.loc[df.ssl_method=='dino', 'ssl_method'] = 'DINO'
df.loc[df.ssl_method=='simclr', 'ssl_method'] = 'SimCLR'
df.loc[df.ssl_method.isna(), 'ssl_method'] = 'Supervised'

sort_methods = {
    'Supervised': 0,
    'SimCLR': 1,
    'DINO': 2,
}

In [ ]:
# sorted(df_ckpts.model_id.unique())

In [ ]:
# df_ssl.model_id.unique()

## Visualize

#### Plotting settings

In [ ]:
linewidth = 3.0
fig_multiplier = 0.7
fig_multiplier = 1
figsize = (10, 16)
figsize = (10, 10)
figsize = (fig_multiplier * figsize[0], fig_multiplier * figsize[1])


color_palette = COLOR_PALETTES['models']
# color_palette_regions = COLOR_PALETTES['regions']
color_palette = color_palette[::2]

In [ ]:

fig, axes = plt.subplots(3, 1, figsize=figsize, dpi=300)
ax = axes[0]

df_plot = df.copy()
df_plot = df_plot.groupby(['model_id','n_samples', 'dataset', 'arch', 'arch_family', 'seed', 'ssl_method']).agg({'score':'mean'}).reset_index()
df_plot['sort'] = df_plot['ssl_method'].map(sort_methods)
df_plot.sort_values(['sort'], inplace=True)
# dataset_sort = {k:i for i, k in enumerate(dataset_pretty.values())}
# Sort as in dataset_pretty
# data = data.sort_values('dataset', key=lambda x: x.map(dataset_sort))

# data['score'] = 1 - data['score']

data_plot = df_plot.rename(columns={'dataset':'Dataset', 'arch_family':'Architecture'})
# data_plot.sort_values('Architecture', inplace=True)
sns.barplot(data=data_plot, x='arch', y='score', hue='ssl_method', ax=ax, palette=color_palette)

ax.set_xlabel('Model', fontsize=16, fontweight='bold')
ax.set_ylabel('Alignment Score (S)', fontsize=16, fontweight='bold')
ax.set_title(f'Effect of Learning Paradigms', fontsize=20, fontweight='bold')
# ax.set_title('ResNet18 Aligment Across Datasets', fontsize=20, fontweight='bold')
# ax.set_ylabel('')
ax.legend(loc='upper right', title='Training Method')
ax.get_legend().remove()
ax.set_ylim([0.3, 0.45])
ax.grid(True)


# ax.plot(compute, pred_losses)
ax.spines[['right', 'top']].set_visible(False)
# ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha='right')
labels = ax.get_xticklabels()

ax.set_xticks(ax.get_xticks(), labels, rotation=30, ha='right', rotation_mode='anchor')
ax.set_ylim([0.1, 0.61])
    


df_plot = df.copy()
for idx, model_arch in enumerate(['ViT-S', 'ResNet-50']):
    ax = axes[idx+1]
    data = df_plot[df_plot.arch == model_arch]


    data_plot = data.rename(columns={'dataset':'Dataset', 'arch_family':'Architecture'})
    data_plot['sort'] = data_plot['region'].map({
        'V1': 0,
        'V2': 1,
        'V4': 2,
        'IT': 3,
        'Behavioral': 4,
    })
    data_plot['sort_methods'] = data_plot['ssl_method'].map(sort_methods)
    data_plot.sort_values(['sort_methods', 'sort'], inplace=True)
    sns.barplot(data=data_plot, x='region', y='score', hue='ssl_method', ax=ax, palette=color_palette)

    ax.set_xlabel('Region', fontsize=16, fontweight='bold')
    ax.set_ylabel('Alignment Score (S)', fontsize=16, fontweight='bold')
    ax.set_title(f'{model_arch} Aligment', fontsize=20, fontweight='bold')
    # ax.set_ylabel('')
    ax.legend(loc='upper left', title='Training Method')
    ax.grid(True)

    # ax.plot(compute, pred_losses)
    ax.spines[['right', 'top']].set_visible(False)
    # ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha='right')
    labels = ax.get_xticklabels()
    ax.set_ylim([0.1, 0.61])

    ax.set_xticks(ax.get_xticks(), labels, rotation=30, ha='right', rotation_mode='anchor')
    
    if idx == 0:
        ax.legend().remove()

plt.tight_layout()


figures_dir = '../figures'
fig_name = 'fig12_unsup_bar'
formats = ['pdf', 'png', 'svg']
save_figs(figures_dir, fig_name, formats=formats)